In [1]:
from pymongo import MongoClient
import pandas as pd
import pickle

In [2]:
json_normalize = pd.json_normalize

In [3]:
import pandas as pd
from itertools import chain
import numpy as np

from bson import json_util, ObjectId

import json

In [4]:
client = MongoClient()
client.list_database_names()

['admin', 'config', 'local', 'ted']

In [5]:
db = client.ted
db.list_collection_names()

['ted_stats_new',
 'ted_stats',
 'ted_comments_new',
 'ted_transcript_new',
 'ted_comments_old',
 'ted_transcript_old']

In [6]:
cursor = db.ted_comments_new.find({}, {'_id':0, 'items.snippet':1})

In [7]:
cursor

In [8]:
df =  pd.DataFrame(list(cursor))

In [9]:

df.head()

,items
0,[{'snippet': {'channelId': 'UCAuUUnT6oDeKwE6v1...
1,[{'snippet': {'channelId': 'UCAuUUnT6oDeKwE6v1...
2,[{'snippet': {'channelId': 'UCAuUUnT6oDeKwE6v1...
3,[{'snippet': {'channelId': 'UCAuUUnT6oDeKwE6v1...
4,[{'snippet': {'channelId': 'UCAuUUnT6oDeKwE6v1...


In [10]:
new_df = pd.concat([pd.DataFrame(json_normalize(x)) for x in df['items']])

In [11]:
new_df.columns

Index(['snippet.channelId', 'snippet.videoId', 'snippet.topLevelComment.kind',
       'snippet.topLevelComment.etag', 'snippet.topLevelComment.id',
       'snippet.topLevelComment.snippet.channelId',
       'snippet.topLevelComment.snippet.videoId',
       'snippet.topLevelComment.snippet.textDisplay',
       'snippet.topLevelComment.snippet.textOriginal',
       'snippet.topLevelComment.snippet.authorDisplayName',
       'snippet.topLevelComment.snippet.authorProfileImageUrl',
       'snippet.topLevelComment.snippet.authorChannelUrl',
       'snippet.topLevelComment.snippet.authorChannelId.value',
       'snippet.topLevelComment.snippet.canRate',
       'snippet.topLevelComment.snippet.viewerRating',
       'snippet.topLevelComment.snippet.likeCount',
       'snippet.topLevelComment.snippet.publishedAt',
       'snippet.topLevelComment.snippet.updatedAt', 'snippet.canReply',
       'snippet.totalReplyCount', 'snippet.isPublic'],
      dtype='object')

In [13]:
new_df.shape

(55605, 21)

In [14]:
new_df_text2 = pd.DataFrame(new_df.groupby(['snippet.videoId'])
                            ['snippet.topLevelComment.snippet.textDisplay'].apply(
                                lambda x: "{%s}" % ', '.join(x)))

In [15]:
new_df_text2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2803 entries, -2Dj9M71JAc to zy2Zj8yIe6c
Data columns (total 1 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   snippet.topLevelComment.snippet.textDisplay  2803 non-null   object
dtypes: object(1)
memory usage: 43.8+ KB


In [16]:
new_df_text2.sample(2)

,snippet.topLevelComment.snippet.textDisplay
snippet.videoId,
KYK6Tfb0snQ,{If humanity could finally understand that rel...
9h2dF-IsH0I,"{YOU NOT TRYING TO RUN !!!, This is an incredi..."


In [17]:
new_df_text2.reset_index(inplace=True)
new_df_text2.rename(columns={'snippet.topLevelComment.snippet.textDisplay':'comments', 
                             'snippet.videoId': 'video_id'}, inplace=True)

In [18]:
with open('ted_video_comments_new.pkl', 'wb') as picklefile:
    pickle.dump(new_df_text2, picklefile)